### Single LGB Model

### Learnings: This notebook is all about different kinds of feature engineering that I could think for this particular problem statement.

### Mistake: Trained on full Train data, should have removed outliers.

In [4]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score,classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import datetime
from time import time
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_columns', None)

In [5]:
df_train = pd.read_excel("train_Data.xlsx")

In [6]:
df_train_bureau = pd.read_excel("train_bureau.xlsx")
df_train_bureau = df_train_bureau.drop_duplicates()

In [7]:
df_test = pd.read_excel("test_Data.xlsx")

In [8]:
df_test_bureau = pd.read_excel("test_bureau.xlsx")
df_test_bureau = df_test_bureau.drop_duplicates()

In [9]:
print("Shape of datasets: ", df_train.shape, df_train_bureau.shape, df_test.shape, df_test_bureau.shape)

Shape of datasets:  (128655, 26) (556980, 25) (14745, 25) (63331, 25)


In [10]:
df_train.head(1)

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months


In [11]:
df_test.head(1)

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE
0,4,Monthly,Advance,Closed,PDC_E,2,GUNA,46,480000,365000.0,365000.0,1000.0,2011-12-29,2015-10-05,2011-12-29,3524747,1046.0,22354,75.83,M,50.0,32069.0,GUNA,MADHYA PRADESH,473001.0


In [12]:
df_total = pd.concat([df_train,df_test],ignore_index=True, sort=False)
print(df_total.shape)

(143400, 26)


In [13]:
df_total_bureau = pd.concat([df_train_bureau,df_test_bureau],ignore_index=True, sort=False)
print(df_total.shape)

(143400, 26)


#### RFM (Recency, Frequency, Monitory) based Features

In [14]:
bureau_pd = df_total_bureau.copy()
demo_pd = df_total.copy()

In [15]:
demo_pd['DisbursalDate'] = demo_pd['DisbursalDate'].apply(lambda x:pd.to_datetime(x))
bureau_pd['DISBURSED-DT'] = bureau_pd['DISBURSED-DT'].apply(lambda x:pd.to_datetime(x))
bureau_pd['DISBURSED-AMT/HIGH CREDIT'] = bureau_pd['DISBURSED-AMT/HIGH CREDIT'].str.replace("[^0-9]","")
bureau_pd['DISBURSED-AMT/HIGH CREDIT'] = bureau_pd['DISBURSED-AMT/HIGH CREDIT'].astype(float)

bureau_pd['CURRENT-BAL'] = bureau_pd['CURRENT-BAL'].str.replace("[^0-9]","")
bureau_pd['CURRENT-BAL'] = bureau_pd['CURRENT-BAL'].astype(float)

merged_df = bureau_pd.merge(demo_pd[['ID','DisbursalDate','DisbursalAmount','Top-up Month']], on=['ID'], how='left')
merged_df = merged_df.sort_values(['ID','DISBURSED-DT'])
merged_df.rename(columns={'DisbursalDate':'Refernce_disb_dt','DisbursalAmount':'ref_disb_amt'}, inplace=True)
merged_df.replace({'ACCOUNT-STATUS' : {'Written Off' : 'other','Suit Filed' : 'other','Settled' : 'other','Restructured' : 'other',
                                      'SUIT FILED (WILFUL DEFAULT)' : 'other','WILFUL DEFAULT' : 'other','Sold/Purchased' : 'other',
                                       'Cancelled' : 'other',}}, inplace= True)

merged_df = pd.get_dummies(merged_df, prefix = ['RFM_acc_status_'], columns=['ACCOUNT-STATUS'])
print(merged_df.shape)
merged_df.head(1)

(620311, 31)


,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,Refernce_disb_dt,ref_disb_amt,Top-up Month,RFM_acc_status__Active,RFM_acc_status__Closed,RFM_acc_status__Delinquent,RFM_acc_status__other
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,2012-02-10,2016-02-01 00:00:00,NaT,NaN,275000.0,NaN,0.0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2012-02-10,275000.0,> 48 Months,0,1,0,0


In [16]:
chk = merged_df.copy()
chk['diff_time'] = (((chk['DISBURSED-DT'] - chk['Refernce_disb_dt']).dt.days/365)*12).round(0)


def bin_loan_return_time(x):
    if x<0:
        return "lr_before_ref"
    if (x>0 and x <= 12):
        return "lr_12_mo"
    elif (x>12 and x<=18):
        return "lr_12_18_mo"
    elif (x>18 and x<=24):
        return "lr_18_24_mo"
    elif (x>24 and x<=30):
        return "lr_24_30_mo"
    elif (x>30 and x<=36):
        return "lr_30_36_mo"
    elif (x>36 and x<=48):
        return "lr_36_48_mo"
    elif x>48:
        return "lr_48_mo"
    
chk['diff_time_bin'] = chk['diff_time'].apply(lambda x: bin_loan_return_time(x))
print(chk.shape)

mg_df1 = merged_df.groupby('ID').agg({'DISBURSED-DT': lambda x: x.min()}).reset_index().rename(columns={'DISBURSED-DT':'min_disb_dt'})
res_df_ffl = merged_df.merge(mg_df1, on='ID', how='left')
res_df_ffl.head(1)

(620311, 33)


,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,Refernce_disb_dt,ref_disb_amt,Top-up Month,RFM_acc_status__Active,RFM_acc_status__Closed,RFM_acc_status__Delinquent,RFM_acc_status__other,min_disb_dt
0,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,2012-02-10,2016-02-01 00:00:00,NaT,NaN,275000.0,NaN,0.0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2012-02-10,275000.0,> 48 Months,0,1,0,0,2012-02-10


In [17]:
res_df_ffl['time_diff_first_loan'] = ((res_df_ffl['DISBURSED-DT'] - res_df_ffl['min_disb_dt']).dt.days/365)*12
res_df_ffl['time_diff_first_loan_bin'] = res_df_ffl['time_diff_first_loan'].apply(lambda x:bin_loan_return_time(x))
print(res_df_ffl.shape)

tmp = res_df_ffl.copy()
tmp['time_diff_first_loan'] = ((tmp['DISBURSED-DT'] - tmp['DISBURSED-DT'].min()).dt.days/365)*12
tmp['time_diff_first_loan_bin'] = tmp['time_diff_first_loan'].apply(lambda x:bin_loan_return_time(x))
tmp = tmp.groupby(['ID','time_diff_first_loan_bin']).agg({'time_diff_first_loan':['mean', 'count','std'], 'DISBURSED-AMT/HIGH CREDIT':['mean','std','sum'], 'CURRENT-BAL':['mean','std','sum'],
                                                   'RFM_acc_status__Active':'sum','RFM_acc_status__Closed':'sum',
                                         'RFM_acc_status__Delinquent':'sum', 'RFM_acc_status__other':'sum'}).reset_index()
tmp.columns = ['ID','time_diff_first_loan_bin_ffl','time_diff_first_loan_mean_ffl','time_diff_first_loan_count_ffl','time_diff_first_loan_std_ffl','disb_amt_mean_ffl',
'disb_amt_std_ffl','disb_amt_sum_ffl','curr_amt_mean_ffl','curr_amt_std_ffl','curr_amt_sum_ffl',
               'RFM_acc_status__Active_ffl' ,'RFM_acc_status__Closed_ffl','RFM_acc_status__Delinquent_ffl','RFM_acc_status__other_ffl']
tmp = tmp.pivot_table(columns=['time_diff_first_loan_bin_ffl'], index = 'ID', values = ['time_diff_first_loan_mean_ffl','time_diff_first_loan_count_ffl','time_diff_first_loan_std_ffl','disb_amt_mean_ffl',
'disb_amt_std_ffl','disb_amt_sum_ffl','curr_amt_mean_ffl','curr_amt_std_ffl','curr_amt_sum_ffl',
               'RFM_acc_status__Active_ffl' ,'RFM_acc_status__Closed_ffl','RFM_acc_status__Delinquent_ffl','RFM_acc_status__other_ffl'], fill_value=0, )
tmp.columns = list(map("_".join, tmp.columns))
tmp = tmp.reset_index()
print(tmp.shape)
tmp.head(1)

(620311, 34)
(141465, 40)


,ID,RFM_acc_status__Active_ffl_lr_12_mo,RFM_acc_status__Active_ffl_lr_36_48_mo,RFM_acc_status__Active_ffl_lr_48_mo,RFM_acc_status__Closed_ffl_lr_12_mo,RFM_acc_status__Closed_ffl_lr_36_48_mo,RFM_acc_status__Closed_ffl_lr_48_mo,RFM_acc_status__Delinquent_ffl_lr_12_mo,RFM_acc_status__Delinquent_ffl_lr_36_48_mo,RFM_acc_status__Delinquent_ffl_lr_48_mo,RFM_acc_status__other_ffl_lr_12_mo,RFM_acc_status__other_ffl_lr_36_48_mo,RFM_acc_status__other_ffl_lr_48_mo,curr_amt_mean_ffl_lr_12_mo,curr_amt_mean_ffl_lr_36_48_mo,curr_amt_mean_ffl_lr_48_mo,curr_amt_std_ffl_lr_12_mo,curr_amt_std_ffl_lr_36_48_mo,curr_amt_std_ffl_lr_48_mo,curr_amt_sum_ffl_lr_12_mo,curr_amt_sum_ffl_lr_36_48_mo,curr_amt_sum_ffl_lr_48_mo,disb_amt_mean_ffl_lr_12_mo,disb_amt_mean_ffl_lr_36_48_mo,disb_amt_mean_ffl_lr_48_mo,disb_amt_std_ffl_lr_12_mo,disb_amt_std_ffl_lr_36_48_mo,disb_amt_std_ffl_lr_48_mo,disb_amt_sum_ffl_lr_12_mo,disb_amt_sum_ffl_lr_36_48_mo,disb_amt_sum_ffl_lr_48_mo,time_diff_first_loan_count_ffl_lr_12_mo,time_diff_first_loan_count_ffl_lr_36_48_mo,time_diff_first_loan_count_ffl_lr_48_mo,time_diff_first_loan_mean_ffl_lr_12_mo,time_diff_first_loan_mean_ffl_lr_36_48_mo,time_diff_first_loan_mean_ffl_lr_48_mo,time_diff_first_loan_std_ffl_lr_12_mo,time_diff_first_loan_std_ffl_lr_36_48_mo,time_diff_first_loan_std_ffl_lr_48_mo
0,1,0,0,3,0,0,5,0,0,1,0,0,0,0,0,68725.111111,0,0,145673.319534,0,0,618526,0,0,244594.666667,0,0,197339.242565,0,0,2201352,0,0,9,0.0,0.0,1391.426484,0,0,34.462006


In [ ]:
tmp1 = chk.groupby('ID')
res_df = pd.DataFrame()
for j in tmp1.groups:
    grp = tmp1.get_group(j)
    grp = grp[['ID','diff_time']]
    grp['loan_freq_diff_lag'] = (grp['diff_time'] - grp['diff_time'].shift(1)).fillna(0.0)
    res_df = pd.concat([res_df,grp])
res_df_time_diff_lag = res_df.groupby('ID').agg({'loan_freq_diff_lag':['mean','std','median']}).reset_index()
res_df_time_diff_lag.columns = ['ID','loan_freq_diff_lag_mean','loan_freq_diff_lag_std','loan_freq_diff_lag_median']

In [18]:
tmp2 = chk.groupby(['ID','diff_time_bin']).agg({'DISBURSED-AMT/HIGH CREDIT':['mean','std','sum'], 'CURRENT-BAL':['mean','std','sum'],
                                        'RFM_acc_status__Active':'sum','RFM_acc_status__Closed':'sum',
                                         'RFM_acc_status__Delinquent':'sum', 'RFM_acc_status__other':'sum'}).reset_index()
tmp2.columns = ['ID','diff_time_bin','disb_amt_mean','disb_amt_std','disb_amt_sum',
               'curr_amt_mean','curr_amt_std','curr_amt_sum','RFM_acc_status__Active' ,'RFM_acc_status__Closed',
              'RFM_acc_status__Delinquent','RFM_acc_status__other']
tmp2 = tmp2.pivot_table(columns=['diff_time_bin'], index = 'ID', values = ['disb_amt_mean','disb_amt_std','disb_amt_sum',
               'curr_amt_mean','curr_amt_std','curr_amt_sum','RFM_acc_status__Active' ,'RFM_acc_status__Closed',
              'RFM_acc_status__Delinquent','RFM_acc_status__other'], fill_value=0, )
tmp2.columns = list(map("_".join, tmp2.columns))
tmp2 = tmp2.reset_index()

In [19]:
chk1 = chk.groupby('ID').agg({'diff_time':lambda x:list(x)}).reset_index()

chk1['lr_before_ref'] = chk1['diff_time'].apply(lambda x:[i for i in x if i<0])
chk1['lr_12_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>0 and i<=12])
chk1['lr_12_18_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>12 and i<=18])
chk1['lr_18_24_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>18 and i<=24])
chk1['lr_24_30_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>24 and i<=30])
chk1['lr_30_36_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>30 and i<=36])
chk1['lr_36_48_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>36 and i<=48])
chk1['lr_48_mo'] = chk1['diff_time'].apply(lambda x:[i for i in x if i>48])

# Recency and Frequency

for num in ['lr_before_ref', 'lr_12_mo', 'lr_12_18_mo',
            'lr_18_24_mo', 'lr_24_30_mo', 'lr_30_36_mo', 'lr_36_48_mo', 'lr_48_mo']:
    chk1[num+'_len'] = chk1[num].apply(lambda x:len(x)).fillna(0)
    chk1[num+'_mean'] = chk1[num].apply(lambda x:np.mean(x))

feat_list_freq = [i for i in chk1.columns if 'mo_len' in i]
feat_list_recency = [i for i in chk1.columns if 'mo_mean' in i]
print(chk1.shape)

(143400, 26)


In [ ]:
df_rfm = res_df_time_diff_lag.merge(tmp2, on='ID', how='left').merge(chk1, on='ID', how='left')

In [ ]:
df_rfm_ffs = df_rfm.merge(tmp, on='ID', how='left')

### Demographic Data Based Feature Engineering

In [16]:
# df_train['ab'] = df_train['Frequency'].copy()
# df_train.replace({'ab' : { 'Half Yearly' : 6, 'Monthly': 1, 'Quatrly': 4, 'BI-Monthly': 2}},inplace = True)
# df_train['nonsense_ratio'] = (df_train['EMI']*df_train['Tenure'])/(df_train['AmountFinance']*df_train['ab'])
# df_train[df_train['nonsense_ratio']<1].shape

In [17]:
# df_total['nonsense_ratio'] = (df_total['EMI']*df_total['Tenure'])/df_total['AmountFinance']
# def bin_nonsense_ratio(x):
#     if x < 1:
#         return "ekdum_not_possible"
#     elif (x>=1 and x<=2):
#         return "possible"
#     elif (x>2 and x<=3):
#         return "slightly maybe"
#     elif (x>3 and x<=4):
#         return "maybee"
#     else:
#         return "fuckall"
    
# df_total['nonsense_ratio_bin'] = df_total['nonsense_ratio'].apply(lambda x: bin_nonsense_ratio(x))

In [121]:
df_total['EMI_to_income_ratio'] = df_total['EMI']/df_total['MonthlyIncome']
df_total['Total_amount_to_pay'] = df_total['EMI']*df_total['Tenure']
df_total['Total_income_within_loan'] = df_total['MonthlyIncome']*df_total['Tenure']

df_total['Interest_amount'] = df_total['Total_amount_to_pay'] - df_total['DisbursalAmount']

df_total['Residual_income'] = df_total['Total_income_within_loan'] - df_total['Total_amount_to_pay']
df_total['Asset_to_total_income'] = df_total['AssetCost']/df_total['Total_income_within_loan']
df_total['Asset_to_monthly_income'] = df_total['AssetCost']/df_total['MonthlyIncome']

df_total['pay_to_loan_ratio'] = df_total['Total_amount_to_pay']/df_total['DisbursalAmount']

df_total['Finance_to_disbursed_ratio'] = df_total['DisbursalAmount']/df_total['AmountFinance']
df_total['Finance_to_disbursed_flag'] = df_total['Finance_to_disbursed_ratio'].apply(lambda x: 1 if x==1 else 0)

In [122]:
# in priority order
df_total["PaymentMode"] = df_total["PaymentMode"].replace("Cheque","PDC")
df_total["PaymentMode"] = df_total["PaymentMode"].replace("Auto Debit","Billed")
df_total["PaymentMode"] = df_total["PaymentMode"].replace("PDC_E","PDC")
df_total["PaymentMode"] = df_total["PaymentMode"].replace("SI Reject","Reject")
df_total["PaymentMode"] = df_total["PaymentMode"].replace("ECS Reject","Reject")
df_total["PaymentMode"] = df_total["PaymentMode"].replace("PDC Reject","Reject")
df_total["PaymentMode"] = df_total["PaymentMode"].replace("Escrow","Direct Debit")

In [123]:
df_total["finance_leverage_money"] = df_total["AssetCost"] - df_total["AmountFinance"]
df_total["finance_leverage_ratio"] = df_total["AssetCost"] / df_total["AmountFinance"]
df_total["loan_amount_ratio"] = df_total["AmountFinance"] / df_total["Tenure"]

df_total["disbursal_leverage_money"] = df_total["AssetCost"] - df_total["DisbursalAmount"]
df_total["disbursal_leverage_ratio"] = df_total["AssetCost"] / df_total["DisbursalAmount"]
df_total["loan_disbursal_ratio"] = df_total["DisbursalAmount"] / df_total["Tenure"]

df_total["processing_fee"] = df_total["AmountFinance"] - df_total["DisbursalAmount"]

df_total["hypothetical_surplus1"] = df_total["loan_amount_ratio"]  - df_total["EMI"]
df_total["hypothetical_surplus2"] = df_total["loan_disbursal_ratio"]  - df_total["EMI"]

df_total["hpothetical_actual_ltv_diff1"] = df_total["finance_leverage_ratio"] - df_total["LTV"]
df_total["hpothetical_actual_ltv_diff2"] = df_total["disbursal_leverage_ratio"] - df_total["LTV"]

df_total["calc_loan"] = df_total["LTV"]*df_total["AssetCost"]
df_total["calc_actual_loan_diff1"] = df_total["LTV"]*df_total["AssetCost"] - df_total["AmountFinance"]
df_total["calc_actual_loan_diff2"] = df_total["LTV"]*df_total["AssetCost"] - df_total["DisbursalAmount"]
df_total["calc_leverage_money"] = df_total["AssetCost"] - df_total["LTV"]*df_total["AssetCost"]

df_total['LTV_BINS']=pd.cut(df_total['LTV'],3,labels=['Low','Medium','High']).astype(np.object)
df_total['asset_value']=df_total['DisbursalAmount']*(df_total['LTV']/100)
df_total['value_cost']=df_total['AssetCost']-df_total['asset_value']
df_total['value_per_cost']=df_total['value_cost']/df_total['asset_value']
df_total['extra_finance']=df_total['AssetCost']*(df_total['LTV']/100)-df_total['DisbursalAmount']
df_total['asset_disburse']=(df_total['AssetCost']-df_total['DisbursalAmount'])/df_total['DisbursalAmount']

In [124]:
df_total['DisbursalDate'] = pd.to_datetime(df_total["DisbursalDate"])
df_total['Disbursal_DOM'] = df_total['DisbursalDate'].dt.day
df_total['Disbursal_DOY'] = df_total['DisbursalDate'].dt.dayofyear
df_total['Disbursal_DOW'] = df_total['DisbursalDate'].dt.dayofweek
df_total['Disbursal_M'] = df_total['DisbursalDate'].dt.month
df_total['Disbursal_Y'] = df_total['DisbursalDate'].dt.year
df_total['Disbursal_WOY'] = df_total['DisbursalDate'].dt.weekofyear

df_total['MaturityDAte'] = pd.to_datetime(df_total["MaturityDAte"])
df_total['Maturity_DOM'] = df_total['MaturityDAte'].dt.day
df_total['Maturity_DOY'] = df_total['MaturityDAte'].dt.dayofyear
df_total['Maturity_DOW'] = df_total['MaturityDAte'].dt.dayofweek
df_total['Maturity_M'] = df_total['MaturityDAte'].dt.month
df_total['Maturity_Y'] = df_total['MaturityDAte'].dt.year
df_total['Maturity_WOY'] = df_total['MaturityDAte'].dt.weekofyear

df_total['AuthDate'] = pd.to_datetime(df_total["AuthDate"])
df_total['Auth_DOM'] = df_total['AuthDate'].dt.day
df_total['Auth_DOY'] = df_total['AuthDate'].dt.dayofyear
df_total['Auth_DOW'] = df_total['AuthDate'].dt.dayofweek
df_total['Auth_M'] = df_total['AuthDate'].dt.month
df_total['Auth_Y'] = df_total['AuthDate'].dt.year
df_total['Auth_WOY'] = df_total['AuthDate'].dt.weekofyear

df_total['time_to_return_loan'] = (df_total['MaturityDAte'] - df_total['DisbursalDate']).dt.days
df_total['time_from_loan_auth'] = (df_total['MaturityDAte'] - df_total['AuthDate']).dt.days
df_total['time_delay_loan_auth_disbursal'] = (df_total['DisbursalDate'] - df_total['AuthDate']).dt.days

In [125]:
def bin_loan_return_time(x):
    if x <= 12:
        return "no_top_up_possible"
    elif (x>12 and x<=18):
        return "less than 18 possible"
    elif (x>18 and x<=24):
        return "less than 24 possible"
    elif (x>24 and x<=30):
        return "less than 30 possible"
    elif (x>30 and x<=36):
        return "less than 36 possible"
    elif (x>36 and x<=48):
        return "less than 48 possible"
    else:
        return "anytime possible"
    
def int_bin_loan_return_time(x):
    if x <= 12:
        return 1
    elif (x>12 and x<=18):
        return 2
    elif (x>18 and x<=24):
        return 3
    elif (x>24 and x<=30):
        return 4
    elif (x>30 and x<=36):
        return 5
    elif (x>36 and x<=48):
        return 6
    else:
        return 7
    
df_total['topup_possible_time1'] = df_total['time_to_return_loan']/30
df_total['topup_possible_time2'] = df_total['time_from_loan_auth']/30

df_total['topup_possible_time1_bin'] = df_total['topup_possible_time1'].apply(lambda x: bin_loan_return_time(x))
df_total['topup_possible_time2_bin'] = df_total['topup_possible_time2'].apply(lambda x: bin_loan_return_time(x))

df_total['int_topup_possible_time1_bin'] = df_total['topup_possible_time1'].apply(lambda x: int_bin_loan_return_time(x))
df_total['int_topup_possible_time2_bin'] = df_total['topup_possible_time2'].apply(lambda x: int_bin_loan_return_time(x))

In [126]:
# df_total['AGE'].fillna(df_total['AGE'].median(),inplace=True) #either fill for rows which have the value present
df_total['age_at_maturity_ifage_at_disbursal'] = df_total['AGE'] + df_total['time_to_return_loan']/365
df_total['age_at_maturity_ifage_at_auth'] = df_total['AGE'] + df_total['time_from_loan_auth']/365

df_total['age_at_maturity_ifage_at_disbursal'] = df_total['AGE'] + df_total['time_to_return_loan']/365
df_total['age_at_maturity_ifage_at_auth'] = df_total['AGE'] + df_total['time_from_loan_auth']/365

# ((pd.to_datetime("2020-01-01") - df_total['MaturityDAte']).dt.days)/365

In [127]:
df_total["monthly_surplus"] = df_total["MonthlyIncome"] - df_total["EMI"]
df_total["income_emi_ratio"] = df_total["MonthlyIncome"] / df_total["EMI"]
df_total["income_amount_ratio1"] = df_total["MonthlyIncome"] / df_total["AmountFinance"]
df_total["income_amount_ratio1"] = df_total["MonthlyIncome"] / df_total["DisbursalAmount"]
df_total["surplus_income_overloan_by_maturity1"] = df_total["MonthlyIncome"]*df_total['topup_possible_time1'] - df_total["AmountFinance"]
df_total["surplus_income_overloan_by_maturity2"] = df_total["MonthlyIncome"]*df_total['topup_possible_time2'] - df_total["AmountFinance"]
df_total["surplus_income_overloan_by_maturity3"] = df_total["MonthlyIncome"]*df_total['topup_possible_time1'] - df_total["DisbursalAmount"]
df_total["surplus_income_overloan_by_maturity4"] = df_total["MonthlyIncome"]*df_total['topup_possible_time2'] - df_total["DisbursalAmount"]

In [128]:
# # columns with high cardinality
# "BranchID" ~ 200
# "Area" ~ 100
# "SupplierID" ~ 4600
# "City" ~ 500
# "ZiPCODE" ~ 9100

In [129]:
df_total['BranchID_SupplierID']=(df_total['SupplierID'].astype(str) + df_total['BranchID'].astype(str)).astype(np.object)
df_total['ZiPCODE_SupplierID']=(df_total['SupplierID'].astype(str) + df_total['ZiPCODE'].astype(str)).astype(np.object)

In [130]:
df_total.replace({'Top-up Month' : { 'No Top-up Service' : 0, '12-18 Months' : 1, '18-24 Months' : 2, '24-30 Months' : 3, '30-36 Months': 4, '36-48 Months': 5, ' > 48 Months':6  }}, inplace= True)

In [131]:
# df_total.drop(["AssetID","DisbursalDate","MaturityDAte","AuthDate","topup_possible_time1","topup_possible_time2"],
#               axis = 1,inplace=True)

### Bureau Data Aggregated Feature Engineering

In [133]:
df_total_bureau.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [134]:
df_total_bureau['CREDIT-LIMIT/SANC AMT'] = df_total_bureau['CREDIT-LIMIT/SANC AMT'].str.replace("[^0-9]","")
df_total_bureau['CREDIT-LIMIT/SANC AMT'] = df_total_bureau['CREDIT-LIMIT/SANC AMT'].astype(float)

df_total_bureau['DISBURSED-AMT/HIGH CREDIT'] = df_total_bureau['DISBURSED-AMT/HIGH CREDIT'].str.replace("[^0-9]","")
df_total_bureau['DISBURSED-AMT/HIGH CREDIT'] = df_total_bureau['DISBURSED-AMT/HIGH CREDIT'].astype(float)

import re
def get_mon(s):
    result = re.sub(r'[^A-Za-z]', '', s)
    return result
df_total_bureau['installment_freq'] = df_total_bureau['INSTALLMENT-AMT'].astype(str).apply(get_mon)
df_total_bureau.replace({'installment_freq' : { 'nan' : 1,'': 1, 'Monthly' : 1,'Quarterly' : 4,'Weekly' : 0.25,'BiWeekly' : 0.5,'Other':1, 'BiMonthly':2, 'Annually':12, 'SinglePaymentLoanbulletballoon': 1, 'Semiannually':6 }},inplace = True)
df_total_bureau['INSTALLMENT-AMT'] = df_total_bureau['INSTALLMENT-AMT'].str.replace("[^0-9]","")
df_total_bureau['INSTALLMENT-AMT'] = df_total_bureau['INSTALLMENT-AMT'].astype(float)
df_total_bureau['INSTALLMENT-AMT_freq'] = df_total_bureau['INSTALLMENT-AMT']/df_total_bureau['installment_freq']

df_total_bureau['CURRENT-BAL'] = df_total_bureau['CURRENT-BAL'].str.replace("[^0-9]","")
df_total_bureau['CURRENT-BAL'] = df_total_bureau['CURRENT-BAL'].astype(float)

df_total_bureau['OVERDUE-AMT'] = df_total_bureau['OVERDUE-AMT'].str.replace("[^0-9]","")
df_total_bureau['OVERDUE-AMT'] = df_total_bureau['OVERDUE-AMT'].astype(float)

df_total_bureau['INSTALLMENT-FREQUENCY_float'] = df_total_bureau['INSTALLMENT-FREQUENCY'].str.replace("[^0-9]","")
df_total_bureau['INSTALLMENT-FREQUENCY_float'] = df_total_bureau['INSTALLMENT-FREQUENCY_float'].astype(float)

In [135]:
df_total_bureau['DATE-REPORTED'] = pd.to_datetime(df_total_bureau["DATE-REPORTED"], errors = 'coerce')
df_total_bureau['DATE-REPORTED_DOM'] = df_total_bureau['DATE-REPORTED'].dt.day
df_total_bureau['DATE-REPORTED_DOY'] = df_total_bureau['DATE-REPORTED'].dt.dayofyear
df_total_bureau['DATE-REPORTED_DOW'] = df_total_bureau['DATE-REPORTED'].dt.dayofweek
df_total_bureau['DATE-REPORTED_M'] = df_total_bureau['DATE-REPORTED'].dt.month
df_total_bureau['DATE-REPORTED_Y'] = df_total_bureau['DATE-REPORTED'].dt.year
df_total_bureau['DATE-REPORTED_WOY'] = df_total_bureau['DATE-REPORTED'].dt.weekofyear

df_total_bureau['DISBURSED-DT'] = pd.to_datetime(df_total_bureau["DISBURSED-DT"], errors = 'coerce')
df_total_bureau['DISBURSED-DT_DOM'] = df_total_bureau['DISBURSED-DT'].dt.day
df_total_bureau['DISBURSED-DT_DOY'] = df_total_bureau['DISBURSED-DT'].dt.dayofyear
df_total_bureau['DISBURSED-DT_DOW'] = df_total_bureau['DISBURSED-DT'].dt.dayofweek
df_total_bureau['DISBURSED-DT_M'] = df_total_bureau['DISBURSED-DT'].dt.month
df_total_bureau['DISBURSED-DT_Y'] = df_total_bureau['DISBURSED-DT'].dt.year
df_total_bureau['DISBURSED-DT_WOY'] = df_total_bureau['DISBURSED-DT'].dt.weekofyear

df_total_bureau['CLOSE-DT'] = pd.to_datetime(df_total_bureau["CLOSE-DT"], errors = 'coerce')
df_total_bureau['CLOSE-DT_DOM'] = df_total_bureau['CLOSE-DT'].dt.day
df_total_bureau['CLOSE-DT_DOY'] = df_total_bureau['CLOSE-DT'].dt.dayofyear
df_total_bureau['CLOSE-DT_DOW'] = df_total_bureau['CLOSE-DT'].dt.dayofweek
df_total_bureau['CLOSE-DT_M'] = df_total_bureau['CLOSE-DT'].dt.month
df_total_bureau['CLOSE-DT_Y'] = df_total_bureau['CLOSE-DT'].dt.year
df_total_bureau['CLOSE-DT_WOY'] = df_total_bureau['CLOSE-DT'].dt.weekofyear

df_total_bureau['LAST-PAYMENT-DATE'] = pd.to_datetime(df_total_bureau["LAST-PAYMENT-DATE"], errors = 'coerce')
df_total_bureau['LAST-PAYMENT-DATE_DOM'] = df_total_bureau['LAST-PAYMENT-DATE'].dt.day
df_total_bureau['LAST-PAYMENT-DATE_DOY'] = df_total_bureau['LAST-PAYMENT-DATE'].dt.dayofyear
df_total_bureau['LAST-PAYMENT-DATE_DOW'] = df_total_bureau['LAST-PAYMENT-DATE'].dt.dayofweek
df_total_bureau['LAST-PAYMENT-DATE_M'] = df_total_bureau['LAST-PAYMENT-DATE'].dt.month
df_total_bureau['LAST-PAYMENT-DATE_Y'] = df_total_bureau['LAST-PAYMENT-DATE'].dt.year
df_total_bureau['LAST-PAYMENT-DATE_WOY'] = df_total_bureau['LAST-PAYMENT-DATE'].dt.weekofyear

df_total_bureau['time_to_pay_installment'] = (df_total_bureau['LAST-PAYMENT-DATE'] - df_total_bureau['DISBURSED-DT']).dt.days
df_total_bureau['time_to_acc_close'] = (df_total_bureau['CLOSE-DT'] - df_total_bureau['DISBURSED-DT']).dt.days
df_total_bureau['time_reprted'] = (df_total_bureau['DATE-REPORTED'] - df_total_bureau['DISBURSED-DT']).dt.days
df_total_bureau['time_close_from_payment'] = (df_total_bureau['CLOSE-DT'] - df_total_bureau['LAST-PAYMENT-DATE']).dt.days
df_total_bureau['time_payment_report'] = (df_total_bureau['LAST-PAYMENT-DATE'] - df_total_bureau['DATE-REPORTED']).dt.days

In [136]:
df_total_bureau['REPORTED DATE - HIST'] = df_total_bureau['REPORTED DATE - HIST'].astype(str)
df_total_bureau['REPORTED DATE - HIST_count'] = df_total_bureau['REPORTED DATE - HIST'].apply(lambda x: len([i for i in x.split(',') if (i!='' and i!='nan')]))

In [159]:
df_total_bureau['DPD - HIST'] = df_total_bureau['DPD - HIST'].str.replace("[^0-9]","")
df_total_bureau['DPD - HIST'] = df_total_bureau['DPD - HIST'].astype(str)
df_total_bureau['DPD - HIST_total_count'] = df_total_bureau['DPD - HIST'].apply(lambda x: len([j for j in [x[i:i+3] for i in range(0, len(x), 3)] if (j!='' and j!='nan')]))
df_total_bureau['DPD - HIST_count_0'] = df_total_bureau['DPD - HIST'].apply(lambda x: len([i for i in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if j!=''] if i=='000']))
df_total_bureau['DPD - HIST_count_not0'] = df_total_bureau['DPD - HIST'].apply(lambda x: len([i for i in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if j!=''] if (i!='000' and i!='nan')]))
df_total_bureau['DPD - HIST_total_sum'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.sum([int(k) for k in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if (j!='' and j!='nan')]]))
df_total_bureau['DPD - HIST_total_mean'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.mean([int(k) for k in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if (j!='' and j!='nan')]]))
df_total_bureau['DPD - HIST_count_not0_mean'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.mean([int(i) for i in [i for i in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if j!=''] if (i!='000' and i!='nan')]]))
df_total_bureau['DPD - HIST_total_median'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.median([int(k) for k in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if (j!='' and j!='nan')]]))
df_total_bureau['DPD - HIST_count_not0_median'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.median([int(i) for i in [i for i in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if (j!='' and j!='nan')] if (i!='000' and i!='nan')]]))
df_total_bureau['DPD - HIST_total_max'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.max([int(k) for k in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if (j!='' and j!='nan')]],initial=0))
df_total_bureau['DPD - HIST_count_not0_max'] = df_total_bureau['DPD - HIST'].apply(lambda x: np.max([int(i) for i in [i for i in [j for j in [x[i:i+3] for i in range(0, len(x), 3)] if j!=''] if (i!='000' and i!='nan')]],initial=0))

In [ ]:
df_total_bureau['CUR BAL - HIST'] = df_total_bureau['CUR BAL - HIST'].astype(str)
df_total_bureau['CUR BAL - HIST_count'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: len([i for i in x.split(',') if i!='']))
df_total_bureau['CUR BAL - HIST_mean'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: np.mean([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['CUR BAL - HIST_median'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: np.median([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['CUR BAL - HIST_min'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: np.min([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]],initial=0))
df_total_bureau['CUR BAL - HIST_max'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: np.max([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]],initial=0))
df_total_bureau['CUR BAL - HIST_std'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: np.std([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['CUR BAL - HIST_sum'] = df_total_bureau['CUR BAL - HIST'].apply(lambda x: np.sum([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))

In [ ]:
df_total_bureau['AMT OVERDUE - HIST'] = df_total_bureau['AMT OVERDUE - HIST'].astype(str)
df_total_bureau['AMT OVERDUE - HIST_count'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: len([i for i in x.split(',') if (i!='' and i!='nan')]))
df_total_bureau['AMT OVERDUE - HIST_mean'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: np.mean([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['AMT OVERDUE - HIST_median'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: np.median([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['AMT OVERDUE - HIST_min'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: np.min([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]],initial=0))
df_total_bureau['AMT OVERDUE - HIST_max'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: np.max([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]],initial=0))
df_total_bureau['AMT OVERDUE - HIST_std'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: np.std([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['AMT OVERDUE - HIST_sum'] = df_total_bureau['AMT OVERDUE - HIST'].apply(lambda x: np.sum([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))

In [ ]:
df_total_bureau['AMT PAID - HIST'] = df_total_bureau['AMT PAID - HIST'].astype(str)
df_total_bureau['AMT PAID - HIST_count'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: len([i for i in x.split(',') if (i!='' and i!='nan')]))
df_total_bureau['AMT PAID - HIST_mean'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: np.mean([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['AMT PAID - HIST_median'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: np.median([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['AMT PAID - HIST_min'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: np.min([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]],initial=0))
df_total_bureau['AMT PAID - HIST_max'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: np.max([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]],initial=0))
df_total_bureau['AMT PAID - HIST_std'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: np.std([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))
df_total_bureau['AMT PAID - HIST_sum'] = df_total_bureau['AMT PAID - HIST'].apply(lambda x: np.sum([float(j) for j in [i for i in str(x).split(',') if (i!='' and i!='nan')]]))

In [ ]:
df_total_bureau['total_malya1'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['OVERDUE-AMT'] + df_total_bureau['WRITE-OFF-AMT'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']

df_total_bureau['total_malya2'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['OVERDUE-AMT'] + df_total_bureau['WRITE-OFF-AMT']
df_total_bureau['total_malya3'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['OVERDUE-AMT'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']
df_total_bureau['total_malya4'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['WRITE-OFF-AMT'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']
df_total_bureau['total_malya5'] = df_total_bureau['OVERDUE-AMT'] + df_total_bureau['WRITE-OFF-AMT'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']

df_total_bureau['total_malya6'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['OVERDUE-AMT']
df_total_bureau['total_malya7'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['WRITE-OFF-AMT']
df_total_bureau['total_malya8'] = df_total_bureau['CURRENT-BAL'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']
df_total_bureau['total_malya9'] = df_total_bureau['OVERDUE-AMT'] + df_total_bureau['WRITE-OFF-AMT']
df_total_bureau['total_malya10'] = df_total_bureau['OVERDUE-AMT'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']
df_total_bureau['total_malya11'] = df_total_bureau['WRITE-OFF-AMT'] + df_total_bureau['DISBURSED-AMT/HIGH CREDIT']

### Getting Bureau data for the same demographic data

In [ ]:
df_total_exp = df_total.copy()
df_total_bureau_exp = df_total_bureau.copy()

In [ ]:
df_total_exp['id_to_join'] = df_total_exp['ID'].astype(str) + "_" + df_total_exp['DisbursalAmount'].astype(str)+ "_" + df_total_exp['DisbursalDate'].astype(str)
df_total_bureau_exp['id_to_join'] = df_total_bureau_exp['ID'].astype(str) + "_" + df_total_bureau_exp['DISBURSED-AMT/HIGH CREDIT'].astype(str) + "_" + df_total_bureau_exp['DISBURSED-DT'].astype(str)
df_total_bureau_exp.drop_duplicates(subset='id_to_join', keep='first', inplace=True)
del df_total_bureau_exp['ID']

print(df_total_exp.shape, df_total_bureau_exp.shape)

data_total_demo_bureau = pd.merge(df_total_exp,df_total_bureau_exp,on='id_to_join',how='left')
print(data_total_demo_bureau.shape)

final_drop_feat = list(set([col for col in data_total_demo_bureau.columns if data_total_demo_bureau[col].nunique()<=1] + ["AssetID","DisbursalDate","MaturityDAte","AuthDate","topup_possible_time1","topup_possible_time2","LAST-PAYMENT-DATE","DATE-REPORTED","DISBURSED-DT","CLOSE-DT","REPORTED DATE - HIST","DPD - HIST","CUR BAL - HIST",
                      "AMT OVERDUE - HIST","AMT PAID - HIST",'id_to_join'] + [col for col in data_total_demo_bureau.columns if data_total_demo_bureau[col].isnull().sum()/data_total_demo_bureau.shape[0] >0.5]))


data_total_demo_bureau.drop(final_drop_feat, axis = 1,inplace=True)

print(data_total_demo_bureau.shape)

In [ ]:
data_total_demo_bureau.head(1)

In [ ]:
df_total_bureau.drop(["LAST-PAYMENT-DATE","DATE-REPORTED","DISBURSED-DT","CLOSE-DT","REPORTED DATE - HIST","DPD - HIST","CUR BAL - HIST",
                      "AMT OVERDUE - HIST","AMT PAID - HIST"] , axis = 1,inplace=True)

# df_total.drop(["AssetID","DisbursalDate","MaturityDAte","AuthDate","topup_possible_time1","topup_possible_time2"],
#               axis = 1,inplace=True)

In [ ]:
df_total_bureau.head(1)

In [ ]:
print(df_total_bureau.shape)

In [ ]:
df_total_bureau.info()

### 1st level aggregations

In [ ]:
df_chk_present = pd.get_dummies(df_total_bureau[['ID','SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS']], 
                                prefix=['SELF-INDICATOR_max','MATCH-TYPE_max','ACCT-TYPE_max','CONTRIBUTOR-TYPE_max','OWNERSHIP-IND_max','ACCOUNT-STATUS_max','INSTALLMENT-FREQUENCY_max','ASSET_CLASS_max'],
                                columns=['SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS'])
df_chk_present = df_chk_present.groupby(['ID']).max().reset_index()
print(df_chk_present.shape)
df_chk_present.head(1)

In [ ]:
df_present_sum = pd.get_dummies(df_total_bureau[['ID','SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS']], 
                                prefix=['SELF-INDICATOR_sum','MATCH-TYPE_sum','ACCT-TYPE_sum','CONTRIBUTOR-TYPE_sum','OWNERSHIP-IND_sum','ACCOUNT-STATUS_sum','INSTALLMENT-FREQUENCY_sum','ASSET_CLASS_sum'],
                                columns=['SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS'])
df_present_sum = df_present_sum.groupby(['ID']).sum().reset_index()
print(df_present_sum.shape)
df_present_sum.head(1)

data_bureau_level1 = pd.merge(df_chk_present,df_present_sum,on='ID',how='left')

In [48]:
num_aggregations = {}
for num in ['SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS','INSTALLMENT-FREQUENCY','ASSET_CLASS']:
    num_aggregations[num] = ['nunique']

id_data_categorical = df_total_bureau.groupby('ID').agg({**num_aggregations})
id_data_categorical.columns = pd.Index(['ID_' + e[0] + "_" + e[1].upper() for e in id_data_categorical.columns.tolist()])
id_data_categorical = id_data_categorical.reset_index()
print(id_data_categorical.shape)
id_data_categorical.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,id_data_categorical,on='ID',how='left')

(143400, 9)


In [49]:
num_aggregations = {}
for num in list(set([col for col in df_total_bureau.columns if df_total_bureau[col].dtype == 'int64' or df_total_bureau[col].dtype == 'float64']) - set(['ID'])):
    num_aggregations[num] = [np.mean, np.min, np.max, np.sum]

id_data_numeric = df_total_bureau.groupby('ID').agg({**num_aggregations})
id_data_numeric.columns = pd.Index(['ID_' + e[0] + "_" + e[1].upper() for e in id_data_numeric.columns.tolist()])
id_data_numeric = id_data_numeric.reset_index()
print(id_data_numeric.shape)
id_data_numeric.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,id_data_numeric,on='ID',how='left')

(143400, 329)


### 2nd level aggregations

In [50]:
df_bureau_sec = df_total_bureau.groupby(['ID','SELF-INDICATOR']).agg({'DISBURSED-AMT/HIGH CREDIT':'sum'}).reset_index().pivot_table(columns=['SELF-INDICATOR'], values=['DISBURSED-AMT/HIGH CREDIT'], index=['ID'], fill_value=0).reset_index()
df_bureau_sec.columns = ['ID'] + ['2ndlevel_'+'SELF-INDICATOR_'+str(i) for i in range(len(df_bureau_sec.columns)-1)]
df_bureau_sec_1 = df_bureau_sec.iloc[:,1:].div(df_bureau_sec.iloc[:,1:].sum(axis=1), axis=0)
df_bureau_sec_1['ID'] = df_bureau_sec['ID'].copy()
print(df_bureau_sec_1.shape)
df_bureau_sec_1.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,df_bureau_sec_1,on='ID',how='left')

(143400, 3)


In [51]:
df_bureau_sec = df_total_bureau.groupby(['ID','ACCT-TYPE']).agg({'DISBURSED-AMT/HIGH CREDIT':'sum'}).reset_index().pivot_table(columns=['ACCT-TYPE'], values=['DISBURSED-AMT/HIGH CREDIT'], index=['ID'], fill_value=0).reset_index()
df_bureau_sec.columns = ['ID'] + ['2ndlevel_'+'ACCT-TYPE_'+str(i) for i in range(len(df_bureau_sec.columns)-1)]
df_bureau_sec_2 = df_bureau_sec.iloc[:,1:].div(df_bureau_sec.iloc[:,1:].sum(axis=1), axis=0)
df_bureau_sec_2['ID'] = df_bureau_sec['ID'].copy()
print(df_bureau_sec_2.shape)
df_bureau_sec_2.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,df_bureau_sec_2,on='ID',how='left')

(143400, 52)


In [52]:
df_bureau_sec = df_total_bureau.groupby(['ID','CONTRIBUTOR-TYPE']).agg({'DISBURSED-AMT/HIGH CREDIT':'sum'}).reset_index().pivot_table(columns=['CONTRIBUTOR-TYPE'], values=['DISBURSED-AMT/HIGH CREDIT'], index=['ID'], fill_value=0).reset_index()
df_bureau_sec.columns = ['ID'] + ['2ndlevel_'+'CONTRIBUTOR-TYPE_'+str(i) for i in range(len(df_bureau_sec.columns)-1)]
df_bureau_sec_3 = df_bureau_sec.iloc[:,1:].div(df_bureau_sec.iloc[:,1:].sum(axis=1), axis=0)
df_bureau_sec_3['ID'] = df_bureau_sec['ID'].copy()
print(df_bureau_sec_3.shape)
df_bureau_sec_3.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,df_bureau_sec_3,on='ID',how='left')

(143400, 13)


In [53]:
df_bureau_sec = df_total_bureau.groupby(['ID','OWNERSHIP-IND']).agg({'DISBURSED-AMT/HIGH CREDIT':'sum'}).reset_index().pivot_table(columns=['OWNERSHIP-IND'], values=['DISBURSED-AMT/HIGH CREDIT'], index=['ID'], fill_value=0).reset_index()
df_bureau_sec.columns = ['ID'] + ['2ndlevel_'+'OWNERSHIP-IND_'+str(i) for i in range(len(df_bureau_sec.columns)-1)]
df_bureau_sec_4 = df_bureau_sec.iloc[:,1:].div(df_bureau_sec.iloc[:,1:].sum(axis=1), axis=0)
df_bureau_sec_4['ID'] = df_bureau_sec['ID'].copy()
print(df_bureau_sec_4.shape)
df_bureau_sec_4.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,df_bureau_sec_4,on='ID',how='left')

(143400, 6)


In [54]:
df_bureau_sec = df_total_bureau.groupby(['ID','ACCOUNT-STATUS']).agg({'DISBURSED-AMT/HIGH CREDIT':'sum'}).reset_index().pivot_table(columns=['ACCOUNT-STATUS'], values=['DISBURSED-AMT/HIGH CREDIT'], index=['ID'], fill_value=0).reset_index()
df_bureau_sec.columns = ['ID'] + ['2ndlevel_'+'ACCOUNT-STATUS_'+str(i) for i in range(len(df_bureau_sec.columns)-1)]
df_bureau_sec_5 = df_bureau_sec.iloc[:,1:].div(df_bureau_sec.iloc[:,1:].sum(axis=1), axis=0)
df_bureau_sec_5['ID'] = df_bureau_sec['ID'].copy()
print(df_bureau_sec_5.shape)
df_bureau_sec_5.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,df_bureau_sec_5,on='ID',how='left')

(143400, 12)


In [55]:
df_curr = df_total_bureau.groupby('ID').agg({'CURRENT-BAL':'sum'})
df_dis = df_total_bureau.groupby(['ID','ACCOUNT-STATUS']).agg({'DISBURSED-AMT/HIGH CREDIT':'sum'}).reset_index()
df_dis = df_dis[df_dis['ACCOUNT-STATUS'] == 'Active']
df_curr = df_curr.merge(df_dis,on='ID',how='left' )
df_curr['current/disbursed_active'] = df_curr['CURRENT-BAL'] / df_curr['DISBURSED-AMT/HIGH CREDIT']
df_curr = df_curr[['ID','current/disbursed_active']]
print(df_curr.shape)
df_curr.head(1)

data_bureau_level1 = pd.merge(data_bureau_level1,df_curr,on='ID',how='left')

(143400, 2)


In [187]:
final_df = pd.merge(data_total_demo_bureau,data_bureau_level1,on='ID',how='left')

data_bureau_level1.shape, data_total_demo_bureau.shape, final_df.shape

((143400, 619), (143400, 149), (143400, 767))

In [188]:
final_df = final_df.merge(df_rfm_ffs[[col for col in df_rfm_ffs.columns if df_rfm_ffs[col].dtype !='object']], on='ID', how='left')
print(final_df.shape)

(143400, 799)


In [189]:
drop_combined_final_feat = list(set([col for col in final_df.columns if final_df[col].nunique()<=1] +  [col for col in final_df.columns if final_df[col].isnull().sum()/final_df.shape[0] >0.4]))

final_df.drop(drop_combined_final_feat, axis = 1,inplace=True)

final_df.shape

(143400, 730)

In [190]:
# for col in final_df.columns:
#     if final_df[col].nunique()>=140000:
#         print(col, ":", final_df[col].nunique())

In [191]:
# for col in final_df.columns:
#     if final_df[col].isnull().sum()/final_df.shape[0]>0.4:
#         print(col, ":", final_df[col].isnull().sum())

In [192]:
# df_total_bureau.replace({'ACCT-TYPE' : { 'Corporate Credit Card' : 'other_kp', 'Loan on Credit Card': 'other_kp', 'Leasing': 'other_kp', 'Business Non-Funded Credit Facility-Priority Sector-Others' : 'other_kp', 'SHG Group': 'other_kp', 'Telco Landline': 'other_kp', 'Staff Loan': 'other_kp'}},inplace = True)
# df_total_bureau.replace({'ASSET_CLASS' : { '1' : 'unknown', '01': 'unknown', '2': 'unknown'}},inplace = True)

In [193]:
final_df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month,EMI_to_income_ratio,Total_amount_to_pay,Total_income_within_loan,Interest_amount,Residual_income,Asset_to_total_income,Asset_to_monthly_income,pay_to_loan_ratio,Finance_to_disbursed_ratio,Finance_to_disbursed_flag,finance_leverage_money,finance_leverage_ratio,loan_amount_ratio,disbursal_leverage_money,disbursal_leverage_ratio,loan_disbursal_ratio,processing_fee,hypothetical_surplus1,hypothetical_surplus2,hpothetical_actual_ltv_diff1,hpothetical_actual_ltv_diff2,calc_loan,calc_actual_loan_diff1,calc_actual_loan_diff2,calc_leverage_money,LTV_BINS,asset_value,value_cost,value_per_cost,extra_finance,asset_disburse,Disbursal_DOM,Disbursal_DOY,Disbursal_DOW,Disbursal_M,Disbursal_Y,Disbursal_WOY,Maturity_DOM,Maturity_DOY,Maturity_DOW,Maturity_M,Maturity_Y,Maturity_WOY,Auth_DOM,Auth_DOY,Auth_DOW,Auth_M,Auth_Y,Auth_WOY,time_to_return_loan,time_from_loan_auth,time_delay_loan_auth_disbursal,topup_possible_time1_bin,topup_possible_time2_bin,int_topup_possible_time1_bin,int_topup_possible_time2_bin,age_at_maturity_ifage_at_disbursal,age_at_maturity_ifage_at_auth,monthly_surplus,income_emi_ratio,income_amount_ratio1,surplus_income_overloan_by_maturity1,surplus_income_overloan_by_maturity2,surplus_income_overloan_by_maturity3,surplus_income_overloan_by_maturity4,BranchID_SupplierID,ZiPCODE_SupplierID,SELF-INDICATOR,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,CURRENT-BAL,OVERDUE-AMT,DATE-REPORTED_DOM,DATE-REPORTED_DOY,DATE-REPORTED_DOW,DATE-REPORTED_M,DATE-REPORTED_Y,DATE-REPORTED_WOY,DISBURSED-DT_DOM,DISBURSED-DT_DOY,DISBURSED-DT_DOW,DISBURSED-DT_M,DISBURSED-DT_Y,DISBURSED-DT_WOY,CLOSE-DT_DOM,CLOSE-DT_DOY,CLOSE-DT_DOW,CLOSE-DT_M,CLOSE-DT_Y,CLOSE-DT_WOY,time_to_acc_close,time_reprted,REPORTED DATE - HIST_count,DPD - HIST_total_count,DPD - HIST_count_0,DPD - HIST_count_not0,DPD - HIST_total_sum,DPD - HIST_total_mean,DPD - HIST_total_median,DPD - HIST_total_max,DPD - HIST_count_not0_max,CUR BAL - HIST_count,CUR BAL - HIST_mean,CUR BAL - HIST_median,CUR BAL - HIST_max,CUR BAL - HIST_std,CUR BAL - HIST_sum,AMT OVERDUE - HIST_count,AMT OVERDUE - HIST_mean,AMT OVERDUE - HIST_median,AMT OVERDUE - HIST_max,AMT OVERDUE - HIST_std,AMT OVERDUE - HIST_sum,total_malya1,total_malya2,total_malya3,total_malya4,total_malya5,total_malya6,total_malya7,total_malya8,total_malya9,total_malya10,total_malya11,SELF-INDICATOR_max_False,SELF-INDICATOR_max_True,MATCH-TYPE_max_PRIMARY,MATCH-TYPE_max_SECONDARY,ACCT-TYPE_max_Auto Loan (Personal),ACCT-TYPE_max_Business Loan - Secured,ACCT-TYPE_max_Business Loan Against Bank Deposits,ACCT-TYPE_max_Business Loan General,ACCT-TYPE_max_Business Loan Priority Sector Agriculture,ACCT-TYPE_max_Business Loan Priority Sector Others,ACCT-TYPE_max_Business Loan Priority Sector Small Business,ACCT-TYPE_max_Business Loan Unsecured,ACCT-TYPE_max_Business Non-Funded Credit Facility General,ACCT-TYPE_max_Business Non-Funded Credit Facility-Priority Sector- Small Business,ACCT-TYPE_max_Business Non-Funded Credit Facility-Priority Sector-Agriculture,ACCT-TYPE_max_Business Non-Funded Credit Facility-Priority Sector-Others,ACCT-TYPE_max_Commercial Equipment Loan,ACCT-TYPE_max_Commercial Vehicle Loan,ACCT-TYPE_max_Construction Equipment Loan,ACCT-TYPE_max_Consumer Loan,ACCT-TYPE_max_Corporate Credit Card,ACCT-TYPE_max_Credit Card,ACCT-TYPE_max_Education Loan,ACCT-TYPE_max_Fleet Card,ACCT-TYPE_max_Gold Loan,ACCT-TYPE_max_Housing Loan,ACCT-TYPE_max_Individual,ACCT-TYPE_max_JLG Group,ACCT-TYPE_max_JLG Individual,ACCT-TYPE_max_Kisan Credit Card,ACCT-TYPE_max_Leasing,ACCT-TYPE_max_Loan Against Bank Deposits,ACCT-TYPE_max_Loan Against Shares / Securities,ACCT-TYPE_max_Loan on Credit Card,ACCT-TYPE_max_Loan to Professional,ACCT-TYPE_max_Microfinance Business Loan,ACCT-TYPE_max_Microfinance Housing Loan,ACCT-TYPE_max_M

### Target Encoding for Multiclass

In [194]:
train_x=final_df[~final_df['Top-up Month'].isnull()].reset_index(drop=True)
train_y=train_x['Top-up Month']
test_x=final_df[final_df['Top-up Month'].isnull()].reset_index(drop=True)

train_x_target_encode = train_x.copy()
test_x_target_encode = test_x.copy()

In [197]:
train_x_target_encode = pd.get_dummies(train_x_target_encode, columns=['Top-up Month'])
print(train_x_target_encode.shape)
train_x_target_encode.head(1)

(128655, 736)


,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,EMI_to_income_ratio,Total_amount_to_pay,Total_income_within_loan,Interest_amount,Residual_income,Asset_to_total_income,Asset_to_monthly_income,pay_to_loan_ratio,Finance_to_disbursed_ratio,Finance_to_disbursed_flag,finance_leverage_money,finance_leverage_ratio,loan_amount_ratio,disbursal_leverage_money,disbursal_leverage_ratio,loan_disbursal_ratio,processing_fee,hypothetical_surplus1,hypothetical_surplus2,hpothetical_actual_ltv_diff1,hpothetical_actual_ltv_diff2,calc_loan,calc_actual_loan_diff1,calc_actual_loan_diff2,calc_leverage_money,LTV_BINS,asset_value,value_cost,value_per_cost,extra_finance,asset_disburse,Disbursal_DOM,Disbursal_DOY,Disbursal_DOW,Disbursal_M,Disbursal_Y,Disbursal_WOY,Maturity_DOM,Maturity_DOY,Maturity_DOW,Maturity_M,Maturity_Y,Maturity_WOY,Auth_DOM,Auth_DOY,Auth_DOW,Auth_M,Auth_Y,Auth_WOY,time_to_return_loan,time_from_loan_auth,time_delay_loan_auth_disbursal,topup_possible_time1_bin,topup_possible_time2_bin,int_topup_possible_time1_bin,int_topup_possible_time2_bin,age_at_maturity_ifage_at_disbursal,age_at_maturity_ifage_at_auth,monthly_surplus,income_emi_ratio,income_amount_ratio1,surplus_income_overloan_by_maturity1,surplus_income_overloan_by_maturity2,surplus_income_overloan_by_maturity3,surplus_income_overloan_by_maturity4,BranchID_SupplierID,ZiPCODE_SupplierID,SELF-INDICATOR,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-AMT/HIGH CREDIT,CURRENT-BAL,OVERDUE-AMT,DATE-REPORTED_DOM,DATE-REPORTED_DOY,DATE-REPORTED_DOW,DATE-REPORTED_M,DATE-REPORTED_Y,DATE-REPORTED_WOY,DISBURSED-DT_DOM,DISBURSED-DT_DOY,DISBURSED-DT_DOW,DISBURSED-DT_M,DISBURSED-DT_Y,DISBURSED-DT_WOY,CLOSE-DT_DOM,CLOSE-DT_DOY,CLOSE-DT_DOW,CLOSE-DT_M,CLOSE-DT_Y,CLOSE-DT_WOY,time_to_acc_close,time_reprted,REPORTED DATE - HIST_count,DPD - HIST_total_count,DPD - HIST_count_0,DPD - HIST_count_not0,DPD - HIST_total_sum,DPD - HIST_total_mean,DPD - HIST_total_median,DPD - HIST_total_max,DPD - HIST_count_not0_max,CUR BAL - HIST_count,CUR BAL - HIST_mean,CUR BAL - HIST_median,CUR BAL - HIST_max,CUR BAL - HIST_std,CUR BAL - HIST_sum,AMT OVERDUE - HIST_count,AMT OVERDUE - HIST_mean,AMT OVERDUE - HIST_median,AMT OVERDUE - HIST_max,AMT OVERDUE - HIST_std,AMT OVERDUE - HIST_sum,total_malya1,total_malya2,total_malya3,total_malya4,total_malya5,total_malya6,total_malya7,total_malya8,total_malya9,total_malya10,total_malya11,SELF-INDICATOR_max_False,SELF-INDICATOR_max_True,MATCH-TYPE_max_PRIMARY,MATCH-TYPE_max_SECONDARY,ACCT-TYPE_max_Auto Loan (Personal),ACCT-TYPE_max_Business Loan - Secured,ACCT-TYPE_max_Business Loan Against Bank Deposits,ACCT-TYPE_max_Business Loan General,ACCT-TYPE_max_Business Loan Priority Sector Agriculture,ACCT-TYPE_max_Business Loan Priority Sector Others,ACCT-TYPE_max_Business Loan Priority Sector Small Business,ACCT-TYPE_max_Business Loan Unsecured,ACCT-TYPE_max_Business Non-Funded Credit Facility General,ACCT-TYPE_max_Business Non-Funded Credit Facility-Priority Sector- Small Business,ACCT-TYPE_max_Business Non-Funded Credit Facility-Priority Sector-Agriculture,ACCT-TYPE_max_Business Non-Funded Credit Facility-Priority Sector-Others,ACCT-TYPE_max_Commercial Equipment Loan,ACCT-TYPE_max_Commercial Vehicle Loan,ACCT-TYPE_max_Construction Equipment Loan,ACCT-TYPE_max_Consumer Loan,ACCT-TYPE_max_Corporate Credit Card,ACCT-TYPE_max_Credit Card,ACCT-TYPE_max_Education Loan,ACCT-TYPE_max_Fleet Card,ACCT-TYPE_max_Gold Loan,ACCT-TYPE_max_Housing Loan,ACCT-TYPE_max_Individual,ACCT-TYPE_max_JLG Group,ACCT-TYPE_max_JLG Individual,ACCT-TYPE_max_Kisan Credit Card,ACCT-TYPE_max_Leasing,ACCT-TYPE_max_Loan Against Bank Deposits,ACCT-TYPE_max_Loan Against Shares / Securities,ACCT-TYPE_max_Loan on Credit Card,ACCT-TYPE_max_Loan to Professional,ACCT-TYPE_max_Microfinance Business Loan,ACCT-TYPE_max_Microfinance Housing Loan,ACCT-TYPE_max_Microfinance O

In [198]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [199]:
for col in [
 'ZiPCODE',
 "BranchID",
 "SupplierID",
 'Area',
 'City',
 'State',
 'topup_possible_time1_bin',
 'BranchID_SupplierID',
 'ZiPCODE_SupplierID']:
    
    trn1, sub1 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_0.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    trn2, sub2 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_1.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    trn3, sub3 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_2.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    trn4, sub4 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_3.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    trn5, sub5 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_4.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    trn6, sub6 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_5.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    trn7, sub7 = target_encode(train_x_target_encode[col], 
                             test_x_target_encode[col], 
                             target=train_x_target_encode['Top-up Month_6.0'], 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)

    train_x[str(col)+'0'] = trn1.values
    train_x[str(col)+'1'] = trn2.values
    train_x[str(col)+'2'] = trn2.values
    train_x[str(col)+'3'] = trn4.values
    train_x[str(col)+'4'] = trn5.values
    train_x[str(col)+'5'] = trn6.values
    train_x[str(col)+'6'] = trn7.values

    test_x[str(col)+'0'] = sub1.values
    test_x[str(col)+'1'] = sub2.values
    test_x[str(col)+'2'] = sub3.values
    test_x[str(col)+'3'] = sub4.values
    test_x[str(col)+'4'] = sub5.values
    test_x[str(col)+'5'] = sub6.values
    test_x[str(col)+'6'] = sub7.values

In [200]:
train_x.shape, test_x.shape

((128655, 793), (14745, 793))

In [205]:
final_df = pd.concat([train_x,test_x], ignore_index=True, sort=False).reset_index(drop=True)
final_df.shape

(143400, 793)

### Modelling

In [206]:
le = LabelEncoder()
for col in final_df.columns:
    if (final_df[col].dtypes == "object"):
        final_df[col].fillna("not_available", inplace = True)
        final_df[col] = final_df[col].astype(str)
        final_df[col] = le.fit_transform(final_df[col])
#         df_total[col] = df_total[col].astype('category')

In [207]:
train_x=final_df[~final_df['Top-up Month'].isnull()].reset_index(drop=True)
train_y=train_x['Top-up Month']
test_x=final_df[final_df['Top-up Month'].isnull()].reset_index(drop=True)

del train_x['Top-up Month']
del test_x['Top-up Month']
test_id = test_x["ID"]
del train_x['ID']
del test_x['ID']

print(train_x.shape, train_y.shape, test_x.shape)

categorical_features_indices = np.where(train_x.dtypes =='category')[0]
print(list(categorical_features_indices))

(128655, 791) (128655,) (14745, 791)
[]


In [208]:
# for col in train_x.columns:
#     if (train_x[col].isnull().sum()/train_x.shape[0])>0.3:
#         print(col,":",train_x[col].isnull().sum())

In [209]:
def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

### LGBM

In [213]:
# from sklearn.utils import class_weight

# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(train_y),
#                                                  train_y)
# {x:y for x,y in zip(np.unique(train_y), class_weights)}

In [218]:
# Final F1 Macro Score: 0.60
NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=18121995)
pred_test = []
pred_x = []
pred_y = []
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    model = LGBMClassifier(n_estimators=20000, random_state=18121995, learning_rate=0.01, objective='multiclass',n_jobs=-1
                          )
    model.fit(train_x.loc[train_ids], train_y.loc[train_ids], 
              eval_set=[(train_x.loc[train_ids], train_y.loc[train_ids]), (train_x.loc[test_ids], train_y.loc[test_ids])],
              verbose=1000,
              categorical_feature="auto",
              early_stopping_rounds=100,
              eval_metric=evaluate_macroF1_lgb)
    pred_fold = model.predict(train_x.loc[test_ids])
    pred_x.extend([int(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_fold_test = model.predict(test_x)
    pred_test.append([int(i) for i  in pred_fold_test])
    print('\n')

print("Final F1 Macro Score:", f1_score(pred_y, pred_x, average='macro'))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

● Fold : 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[677]	valid_0's multi_logloss: 0.218317	valid_0's macroF1: 0.763757	valid_1's multi_logloss: 0.331982	valid_1's macroF1: 0.601176


● Fold : 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[574]	valid_0's multi_logloss: 0.230645	valid_0's macroF1: 0.745381	valid_1's multi_logloss: 0.339903	valid_1's macroF1: 0.59292


● Fold : 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[511]	valid_0's multi_logloss: 0.24119	valid_0's macroF1: 0.73084	valid_1's multi_logloss: 0.339724	valid_1's macroF1: 0.596091


● Fold : 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[504]	valid_0's multi_logloss: 0.242431	valid_0's macroF1: 0.725301	valid_1's multi_logloss: 0.33568	valid_1's macroF1: 0.601403


● Fold : 5
Training until validation scores 

In [219]:
sorted(zip(model.feature_importances_,train_x),reverse=True)

[(2788, 'Disbursal_Y'),
 (2320, '2ndlevel_ACCT-TYPE_20'),
 (1926, 'lr_18_24_mo_amt_mean'),
 (1724, 'lr_24_30_mo_amt_mean'),
 (1617, '2ndlevel_ACCT-TYPE_39'),
 (1410, 'lr_30_36_mo_amt_mean'),
 (1325, 'lr_12_18_mo_amt_mean'),
 (1281, 'lr_36_48_mo_amt_mean'),
 (1097, 'time_reprted'),
 (1045, 'SELF-INDICATOR_sum_True'),
 (1022, 'lr_48_mo_amt_mean'),
 (973, '2ndlevel_ACCT-TYPE_38'),
 (803, 'ZiPCODE1'),
 (736, 'Tenure'),
 (720, 'ID_time_reprted_AMIN'),
 (697, 'State4'),
 (684, 'SupplierID1'),
 (646, 'MonthlyIncome'),
 (639, 'lr_48_mo_len'),
 (634, 'SupplierID4'),
 (631, 'Disbursal_DOY'),
 (624, 'ZiPCODE_SupplierID1'),
 (594, 'ID_DATE-REPORTED_Y_MEAN'),
 (593, 'ZiPCODE4'),
 (585, 'ZiPCODE5'),
 (579, 'State1'),
 (559, 'time_to_return_loan'),
 (559, 'ZiPCODE3'),
 (559, 'State'),
 (544, 'LoanStatus'),
 (518, 'ID_DISBURSED-DT_Y_AMAX'),
 (516, 'ID_DATE-REPORTED_Y_AMAX'),
 (491, 'City1'),
 (482, 'SupplierID3'),
 (465, 'State5'),
 (464, 'ID_AMT OVERDUE - HIST_count_AMAX'),
 (456, 'SupplierID5'),
 (4

In [220]:
from scipy import stats

vote = stats.mode(final_prediction, axis=1)[0].reshape(-1)
final_prediction['Top-up Month'] = vote
final_prediction['ID'] = test_id
print(final_prediction.shape)
final_prediction.head()

(14745, 7)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,Top-up Month,ID
0,0,0,0,0,0,0,4
1,6,6,6,6,6,6,5
2,0,0,0,0,0,0,6
3,0,0,0,0,0,0,25
4,0,0,0,0,0,0,119


In [221]:
final_prediction['Top-up Month'].value_counts(normalize=True)

0    0.934825
6    0.041166
5    0.008884
2    0.004612
4    0.004476
3    0.003798
1    0.002238
Name: Top-up Month, dtype: float64

In [222]:
df_submission = final_prediction[['ID', 'Top-up Month']]
df_submission.replace({'Top-up Month' : { 0: 'No Top-up Service', 1: '12-18 Months', 2: '18-24 Months', 3: '24-30 Months', 4: '30-36 Months', 5: '36-48 Months', 6: ' > 48 Months'}}, inplace= True)
df_submission.to_csv("ltfs3_v2_target_encoding.csv", index = False)
df_submission.head()

,ID,Top-up Month
0,4,No Top-up Service
1,5,> 48 Months
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service
